In [1]:
%load_ext rich

# LLM Provider
A provider is a class that implements `ProviderProtocol`, serving as an interface to interact with a large language model (LLM). It processes user queries and returns model-generated responses.

```python
class LLMProtocol(Protocol):
    async def achat(self, history: list[Message]) -> Message: ...
```

Expected Behavior

-  Implements `achat(self, history: list[Message]) -> Message`
-  Accepts a conversation history (list of `Message` objects)
-  Returns a single assistant-generated response (`Message`)

A provider facilitates structured communication with an LLM, ensuring consistency in message handling and response generation.

In [3]:
from promptimus import Message, MessageRole
from promptimus.llms import OpenAILike

In [4]:
# creating a provider
#
# do not forgent to export OPENAI_API_KEY with your token or pass it as api-key argument
llm = OpenAILike(
    model_name="gemma3:4b", base_url="http://lilan:11434/v1", api_key="DUMMY"
)

In [5]:
# using a provider with single message
await llm.achat(
    [
        Message(role=MessageRole.USER, content="Hi, who are you?"),
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="Hi there! I'm Gemma, a large language model created by the Gemma team at Google DeepMind. I'm an open-weights model, meaning I’m widely available for public use! \n\nI can take text and images as inputs and generate text-based responses. \n\nYou can learn more about me on the Gemma website: [https://gemma.google.com/](https://gemma.google.com/) \n\nHow can I help you today?",
    tool_calls=None,
    tool_call_id=None
)

In [6]:
# using provider with multiple messages
await llm.achat(
    [
        Message(role=MessageRole.SYSTEM, content="Your name is Mark."),
        Message(role=MessageRole.USER, content="Hi, what is your name?"),
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='Hi there! My name is Mark. It’s nice to meet you! 😊 \n\nHow can I help you today?',
    tool_calls=None,
    tool_call_id=None
)

## OpenAI like and rate limits

In [7]:
import asyncio

from tqdm.auto import tqdm

In [8]:
openai = OpenAILike(
    model_name="gpt-4.1-nano",
    max_concurrency=50,
)

In [9]:
for task in tqdm.as_completed(
    [
        openai.achat(
            [
                Message(role=MessageRole.USER, content="Hi, who are you?"),
            ]
        )
        for _ in range(1_000)
    ]
):
    await task

  0%|          | 0/1000 [00:00<?, ?it/s]

2025-06-18 08:58:45.859 | WARNING  | promptimus.common.rate_limiting:execute_request:54 - OpenAILike rate limit hit (attempt 1/5), waiting 3.000s (base 3.000s, exponential backoff)
2025-06-18 08:58:53.300 | WARNING  | promptimus.common.rate_limiting:execute_request:54 - OpenAILike rate limit hit (attempt 2/5), waiting 6.000s (base 3.000s, exponential backoff)
2025-06-18 08:59:00.150 | WARNING  | promptimus.common.rate_limiting:execute_request:54 - OpenAILike rate limit hit (attempt 3/5), waiting 12.000s (base 3.000s, exponential backoff)
2025-06-18 08:59:12.747 | INFO     | promptimus.common.rate_limiting:execute_request:47 - OpenAILike rate limit resolved.
2025-06-18 08:59:12.815 | INFO     | promptimus.common.rate_limiting:execute_request:47 - OpenAILike rate limit resolved.
2025-06-18 08:59:14.470 | WARNING  | promptimus.common.rate_limiting:execute_request:54 - OpenAILike rate limit hit (attempt 4/5), waiting 24.000s (base 3.000s, exponential backoff)
2025-06-18 08:59:39.045 | INFO

# Embedder
A embedder is a class that implements `EmbedderProtocol`, serving as an interface to generate text embeddings.

```python
Embedding = list[float]


class EmbedderProtocol(Protocol):
    async def aembed_batch(self, texts: list[str], **kwargs: Any) -> list[Embedding]: ...
    async def aembed(self, text: str, **kwargs: Any) -> Embedding: ...
```

In [10]:
import promptimus as pm

In [11]:
embedder = pm.embedders.OpenAILikeEmbedder(
    model_name="mxbai-embed-large", base_url="http://lilan:11434/v1", api_key="DUMMY"
)

In [12]:
embedding = await embedder.aembed("Hi")
len(embedding), embedding[:3]

(1024, [0.049403064, 0.012764112, -0.006060004])

In [13]:
embeddings = await embedder.aembed_batch(["Hi", "How are you?"])

len(embeddings)

2